In [1]:
%pip install gradio --quiet
%pip install xformer --quiet
%pip install chromadb --quiet
%pip install langchain --quiet
%pip install accelerate --quiet
%pip install transformers --quiet
%pip install bitsandbytes --quiet
%pip install unstructured --quiet
%pip install sentence-transformers --quiet
%pip install -U faiss-gpu tiktoken
%pip install langchainhub --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account

In [11]:
# %pip uninstall -y transformers
# %pip install transformers --quiet
!pip install --upgrade transformers

In [2]:
import random
import torch
import os
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
import pandas as pd
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import torch
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain import hub
from IPython.display import Markdown, display
import time


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [9]:
class MedicalChatbot:
    def __init__(self, embedding_model_name='thenlper/gte-base', llm_model_name="mistralai/Mistral-7B-Instruct-v0.1", faiss_indices_path="../data/faiss_indices"):
        self.embedding_model_name = embedding_model_name
        self.llm_model_name = llm_model_name
        self.faiss_indices_path = faiss_indices_path
        self.device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_built() else "cpu"
        # self.init_embedding_model()
        # self.load_FAISS_DB_retriever()
        # self.init_tokenizer()
        # self.init_model()
        # self.init_pipeline()
        # self.init_llm()
        # self.init_prompt_template()
        # self.init_qa_chain()


    def generate_response(self, user_query):
        result = self.qa_chain({"query": user_query})
        return result

    def init_embedding_model(self):
        model_kwargs = {'device': self.device}
        encode_kwargs = {'normalize_embeddings': True}
        self.embed_model = HuggingFaceEmbeddings(model_name=self.embedding_model_name, model_kwargs=model_kwargs,
                                            encode_kwargs=encode_kwargs)

    def load_FAISS_DB_retriever(self):
        self.faiss_db = FAISS.load_local(f"{self.faiss_indices_path}/{self.embedding_model_name}", self.embed_model)
        self.faiss_retriever = self.faiss_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})


    def get_relevant_documents_for_query(self, query):
        return self.faiss_retriever.invoke(query)

    def init_model(self):
        if self.device == "cuda":
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True
            )
            self.model=AutoModelForCausalLM.from_pretrained(
            self.llm_model_name, torch_dtype=torch.float16,
            trust_remote_code=True,
            device_map="auto",
            quantization_config=quantization_config
            )

        else:
            self.model = AutoModelForCausalLM.from_pretrained(self.llm_model_name, torch_dtype="auto",  trust_remote_code=True)



    def init_tokenizer(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.llm_model_name, use_fast=True)
        # self.tokenizer.pad_token = self.tokenizer.eos_token

    def init_pipeline(self):
        generation_config = GenerationConfig.from_pretrained(self.llm_model_name)
        generation_config.max_new_tokens = 512
        generation_config.temperature = 0.001
        generation_config.top_p = 0.95
        generation_config.do_sample = True
        generation_config.repetition_penalty = 1.15

        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            return_full_text=True,
            device=None if self.device=="cuda" else self.device ,
            generation_config=generation_config,
    )

    def init_llm(self):
        self.llm = HuggingFacePipeline(
            pipeline=self.pipeline,
    )


    def init_prompt_template(self):

        # template = """Use the following pieces of context to answer the question at the end.
        # If you don't know the answer, just say that you don't know, don't try to make up an answer.
        # Use three sentences maximum and keep the answer as concise as possible.
        # Always say "thanks for asking!" at the end of the answer.
        #
        # {context}
        #
        # Question: {question}
        #
        # Helpful Answer:"""
        # self.custom_rag_prompt = PromptTemplate.from_template(template)

        self.prompt  = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

    def init_qa_chain(self):
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.faiss_retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": self.prompt},
        )

    def init_all(self):
        self.init_embedding_model()
        self.load_FAISS_DB_retriever()
        self.init_tokenizer()
        self.init_model()
        self.init_pipeline()
        self.init_llm()
        self.init_prompt_template()
        self.init_qa_chain()

In [14]:
import time
from IPython.display import Markdown, display

def generate_response_and_measure_time(query, chatbot):
    start_time = time.time()  # Start timing
    result = chatbot.generate_response(query)
    execution_time = time.time() - start_time  # Calculate execution time

    # Print the execution time rounded to 2 decimal places
    print(f"The execution time on {chatbot.device} using {chatbot.llm_model_name} is {round(execution_time, 2)} seconds.\n")

    # Display the query and result
    display(Markdown(f"<b>{result['query']}</b>"))
    display(Markdown(f"<p>{result['result']}</p>"))



## Testing on a local device with MPS:

The accelerator and quantization support only CUDA, so MPS is much slower than CUDA.
- "<b>microsoft/phi-2</b>" works and takes approximately 45 seconds per request.
- "<b>mistralai/Mistral-7B-Instruct-v0.1</b>" does not work due to the error "TypeError: BFloat16 is not supported on MPS."

In [ ]:
chatbot = MedicalChatbot(llm_model_name="microsoft/phi-2")

In [ ]:
chatbot.device

'mps'

In [ ]:
chatbot.init_all()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
query = "Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The execution time on mps using microsoft/phi-2 is 46.27 seconds.


<b>Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?</b>

<p> No, occupational outcome in bipolar disorder is not predicted by premorbid functioning and intelligence. While studies have shown that individuals with bipolar disorder often experience difficulties in their careers due to the nature of the condition, it appears that other factors play a larger role in determining employment status. These include the severity of the individual's symptoms, the type of job they hold, and whether or not they receive appropriate treatment. Additionally, socioeconomic factors such as income and education level can also influence one’s chances of finding stable employment. Therefore, while premorbid functioning and intelligence may contribute to overall success in life, these two variables alone do not appear to be sufficient indicators of future career prospects for someone living with bipolar disorder.</p>

In [ ]:
query = "What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The execution time on mps using microsoft/phi-2 is 46.25 seconds.


<b>What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?</b>

<p> The effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman are not well established. While some studies have suggested that AAT treatment can improve symptoms of CFS, others have found no significant difference between AAT treatment and placebo. Therefore, it is difficult to make definitive conclusions about the effectiveness of AAT treatment for CFS based on this single case study alone. Further research is needed to determine whether AAT treatment is beneficial for individuals with CFS.
</p>

In [ ]:
chatbot = MedicalChatbot(llm_model_name="mistralai/Mistral-7B-Instruct-v0.1")

In [ ]:
chatbot.device

'mps'

In [ ]:
chatbot.init_all()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: BFloat16 is not supported on MPS

In [ ]:
query = "Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?"
generate_response_and_measure_time(query, chatbot)

In [10]:
chatbot = MedicalChatbot(llm_model_name="microsoft/phi-2", faiss_indices_path="faiss_indices")

In [11]:
chatbot.device

'cuda'

In [12]:
chatbot.init_all()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
query = "Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The execution time on cuda using microsoft/phi-2 is 6.11 seconds.


<b>Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?</b>

<p> No, occupational outcome in bipolar disorder is not predicted by premorbid functioning and intelligence.</p>

In [16]:
query = "What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The execution time on cuda using microsoft/phi-2 is 5.8 seconds.


<b>What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?</b>

<p> According to the provided context, there is no direct information about the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS). Therefore, it cannot be determined whether AAT treatment can alleviate the symptoms of CFS based solely on this context. Further research would be needed to determine the effectiveness of AAT treatment on CFS.
</p>

In [20]:
chatbot = MedicalChatbot(llm_model_name="mistralai/Mistral-7B-Instruct-v0.1",  faiss_indices_path="faiss_indices")

In [21]:
chatbot.device

'cuda'

In [22]:
chatbot.init_all()

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [23]:
query = "Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The execution time on cuda using mistralai/Mistral-7B-Instruct-v0.1 is 3.11 seconds.


<b>Is occupational outcome in bipolar disorder predicted by premorbid functioning and intelligence?</b>

<p> No, occupational outcome in bipolar disorder is not predicted by premorbid functioning and intelligence.</p>

In [24]:
query = "What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The execution time on cuda using mistralai/Mistral-7B-Instruct-v0.1 is 8.96 seconds.


<b>What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?</b>

<p> Based on the case study, α1-antitrypsin (AAT) treatment showed improvements in maximal workload, working memory scores, and perceptual organization scores in a 49-year-old woman with chronic fatigue syndrome (CFS). The patient also experienced an improvement in functional capacity, allowing her to work part-time employment. These findings suggest a possible role for AAT in the treatment of CFS.</p>

In [25]:
query = "Is there a cure to cancer?"
generate_response_and_measure_time(query, chatbot)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The execution time on cuda using mistralai/Mistral-7B-Instruct-v0.1 is 5.72 seconds.


<b>Is there a cure to cancer?</b>

<p> No, there isn't a cure to cancer yet. Current treatment strategies aim at reducing the tumor burden and managing symptoms, but cancer often recurs. Newer therapies like gene editing and AI-powered treatments are being explored, but a definitive cure remains elusive.</p>

In [29]:
query = "Is there a cure to cancer?"
result = chatbot.generate_response(query)
display(Markdown(f"<b>{result['query']}</b>"))
display(Markdown(f"<p>{result['result']}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Is there a cure to cancer?</b>

<p> No, there isn't a cure to cancer yet. Current treatment strategies aim at reducing the tumor burden and managing symptoms, but cancer often recurs. Newer therapies like gene editing and AI-powered treatments are being explored, but a definitive cure remains elusive.</p>

In [32]:
display(Markdown(f"<b>Source Documents:</b>"))
display(Markdown(f"<p>{result['source_documents']}</p>"))


<b>Source Documents:</b>

<p>[Document(page_content='Abstract: The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellbeing In an effort to determine whether we are achieving this last goal in oncology herein we review the biological and clinical framework that has led to the foundations of the current anticancer treatment paradigm Currently cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure This strategy has led to a substantial improvement in survival rates as cancer mortality rates have dropped in an unprecedented way Despite this progress more than 9 million people still die from cancer every year indicating that the current treatment strategy is not leading to a cancer cure but to a cancer remission that is the temporary absence of manifestations of a particular disease after months or years of remission in most patients cancer will inevitably recur Our critical analysis indicates that it is time to discuss about the new key challenges and future directions in clinical oncology We need to generate novel treatment strategies more suited to the current clinical reality\nAbstract_Length: 1215', metadata={'source': 'processed_data_part1.csv', 'row': 15208, 'PMID': '33312104', 'Title': 'Why we do what we do. A brief analysis of cancer therapies.', 'Authors': 'Carlos M Galmarini', 'Publication Date': '2020', 'DOI': '10.17179/excli2020-2972'}), Document(page_content='Abstract: Cancer is becoming one of the deadliest disease in both developed as well as developing countries and continuous effort is being made to find innovative therapies for myriad types of cancers that afflict the human body Therapeutic options for cancer have grown exponentially over the time but we are quite a way off from finding a magic bullet that can help cure cancer and based on the current evidence we may never find a catch all cure ever and it becomes crucial that we keep on innovating and find multiple ways to attack the menace of this dreaded disease Many patients suffer recurrence of disease and require secondline or in some cases more than two lines of treatment In this review article we have discussed the available therapies along with the newer advancements that have been made in cancer therapy Latest developments in treatment of various cancers that have been discussed include gene editing using CRISPRCas9 theranostics viral mediated therapy artificial intelligence tumor infiltrating lymphocyte therapy etc\nAbstract_Length: 1031', metadata={'source': 'processed_data_part1.csv', 'row': 28024, 'PMID': '34411650', 'Title': 'Futuristic approach to cancer treatment.', 'Authors': 'Deepti Malik; Saniya Mahendiratta; Harpinder Kaur; Bikash Medhi', 'Publication Date': '2021-Dec-30', 'DOI': '10.1016/j.gene.2021.145906'}), Document(page_content='232 7 rhetoric pertaining to the latest technology n63 203 and 8 focusing treatment on cancer origins and cause n43 139 Overall 258 n80 of advertisements included a direct statement claiming provider treatment can cure cancer or prolong life Our results provide evidence alternative cancer providers are using Meta advertising products to market scientifically unsupported cancer treatments Advertisements regularly referenced alternative and natural treatment approaches to cancer Imagery and text content that emulated evidencebased medical providers created the impression that the offered treatments were effective medical options for cancer Advertisements exploited the hope of patients with terminal and poor prognoses by sharing testimonials of past patients who allegedly were cured or had their lives prolonged We recommend that Meta introduce a mandatory humanled authorization process that is not reliant upon artificial intelligence for medicalrelated advertisers before giving advertising permissions Further research should focus on the conflict of interest between social media platforms advertising products and public health', metadata={'source': 'processed_data_part2.csv', 'row': 20798, 'PMID': '37256649', 'Title': 'Advertising Alternative Cancer Treatments and Approaches on Meta Social Media Platforms: Content Analysis.', 'Authors': 'Marco Zenone; Jeremy Snyder; JeanChristophe BelislePipon; Timothy Caulfield; May van Schalkwyk; Nason Maani', 'Publication Date': '2023-May-31', 'DOI': '10.2196/43548'})]</p>

In [53]:
[r.metadata for r in result['source_documents']]

[{'source': 'processed_data_part1.csv',
  'row': 15208,
  'PMID': '33312104',
  'Title': 'Why we do what we do. A brief analysis of cancer therapies.',
  'Authors': 'Carlos M Galmarini',
  'Publication Date': '2020',
  'DOI': '10.17179/excli2020-2972'},
 {'source': 'processed_data_part1.csv',
  'row': 28024,
  'PMID': '34411650',
  'Title': 'Futuristic approach to cancer treatment.',
  'Authors': 'Deepti Malik; Saniya Mahendiratta; Harpinder Kaur; Bikash Medhi',
  'Publication Date': '2021-Dec-30',
  'DOI': '10.1016/j.gene.2021.145906'},
 {'source': 'processed_data_part2.csv',
  'row': 20798,
  'PMID': '37256649',
  'Title': 'Advertising Alternative Cancer Treatments and Approaches on Meta Social Media Platforms: Content Analysis.',
  'Authors': 'Marco Zenone; Jeremy Snyder; JeanChristophe BelislePipon; Timothy Caulfield; May van Schalkwyk; Nason Maani',
  'Publication Date': '2023-May-31',
  'DOI': '10.2196/43548'}]